# RNN으로_텍스트_분석하기
* RNN: 순환신경망
* 언어, 시계열 데이터 분석에 주로 사용
* 길이가 길어지면 기울기 소실 문제  방생
* LSTM, GRU 

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np

2025-05-22 10:08:33.176110: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-22 10:08:33.189047: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747876113.203251    2090 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747876113.207643    2090 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747876113.220635    2090 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# 텐서플로에서 텍스트 전처리 하기
* 토큰화 : 문장을 단어 혹은 형태소로 쪼개는 것
* 원핫인코딩 : 문자를 벡터화
* 임베딩(enbedding):백터화 -> 원핫인코딩을 더 축소

In [2]:
text = "해보지 않으면 해낼수 없다"

In [3]:
result = text_to_word_sequence(text)
result

['해보지', '않으면', '해낼수', '없다']

단어 빈도수 세기

In [4]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화 합니다',
       '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다',
       '토큰화한 결과는 딥러닝에서 사용할 수 있습니다',
       '텍스트 전처리에는 벡터화 원핫인코딩 패딩으로 길이 맞추기 등이 필요합니다',
       '딥러닝 쉽지 않네요']

In [5]:
token = Tokenizer()
token.fit_on_texts(docs)
print("단어카운트 :", token.word_counts, end="\n\n")
print("문장카운트 :", token.document_count, end="\n\n")
print('각 단어가 몇 개의 문장에 포함되어있는지 계산 :', token.word_docs, end="\n\n")
print("각 단어의 매겨진 인덱스값: ", token.word_index, end="\n\n")

단어카운트 : OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화', 1), ('합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1), ('텍스트', 1), ('전처리에는', 1), ('벡터화', 1), ('원핫인코딩', 1), ('패딩으로', 1), ('길이', 1), ('맞추기', 1), ('등이', 1), ('필요합니다', 1), ('딥러닝', 1), ('쉽지', 1), ('않네요', 1)])

문장카운트 : 5

각 단어가 몇 개의 문장에 포함되어있는지 계산 : defaultdict(<class 'int'>, {'나누어': 1, '텍스트의': 2, '각': 1, '토큰화': 1, '합니다': 1, '먼저': 1, '단어를': 1, '토큰화해야': 1, '딥러닝에서': 2, '단어로': 1, '인식됩니다': 1, '있습니다': 1, '수': 1, '사용할': 1, '토큰화한': 1, '결과는': 1, '등이': 1, '패딩으로': 1, '텍스트': 1, '맞추기': 1, '원핫인코딩': 1, '전처리에는': 1, '필요합니다': 1, '길이': 1, '벡터화': 1, '쉽지': 1, '딥러닝': 1, '않네요': 1})

각 단어의 매겨진 인덱스값:  {'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화': 7, '합니다': 8, '단어로': 9, '토큰화해야': 10, '인식됩니다': 11, '토큰화한': 12, '결과는': 13, '사용할': 14, '수': 15, '있습니다': 16, '텍스트': 17, '전처리에는': 18, '벡터화': 19, '원핫인코딩': 20, '패딩으로': 21, '길이': 22, '맞추기': 23, '등이': 

In [6]:
'딥러닝 쉽지 않네요' -> [26, 27, 28]

SyntaxError: invalid syntax (2529750990.py, line 1)

In [7]:
x = token.texts_to_sequences(docs)
print(x)

[[3, 1, 4, 5, 6, 7, 8], [1, 9, 10, 2, 11], [12, 13, 2, 14, 15, 16], [17, 18, 19, 20, 21, 22, 23, 24, 25], [26, 27, 28]]


가장 긴 백터의 길이 추출

In [8]:
max([len(i) for i in x])

9

# 가장 긴 길이의 문장에 맞추어 패딩
* 가장 긴 문장 길이 + 1의 길이로 패딩
* 문장의 시작에는 반드시 0이 있어야 함

In [9]:
padded_x = pad_sequences(x, max([len(i) for i in x])+1)
padded_x

array([[ 0,  0,  0,  3,  1,  4,  5,  6,  7,  8],
       [ 0,  0,  0,  0,  0,  1,  9, 10,  2, 11],
       [ 0,  0,  0,  0, 12, 13,  2, 14, 15, 16],
       [ 0, 17, 18, 19, 20, 21, 22, 23, 24, 25],
       [ 0,  0,  0,  0,  0,  0,  0, 26, 27, 28]], dtype=int32)

In [10]:
padded_x.shape

(5, 10)

# 텍스트를 읽고 긍정, 부정 예측하기

In [11]:
docs2 = ['너무 재미있네요',
        '최고예요',
        '참 신기한 딥러닝이네요',
        '인공지능 칭찬합니다',
        '더 자세히 배우고 싶어요',
        '변화가 너무 빨라요',
        "GPT성능이 생각보다 별로네요",
        '제미나이보다는 낫죠',
        '나는 차라리 라마를 쓴다',
        '유료 결제 싫어요']


In [12]:
classes = np.array([1,1,1,1,1,0,0,1,0,0])

In [13]:
token = Tokenizer()
token.fit_on_texts(docs2)

In [14]:
x = token.texts_to_sequences(docs2)
print(x)

[[1, 2], [3], [4, 5, 6], [7, 8], [9, 10, 11, 12], [13, 1, 14], [15, 16, 17], [18, 19], [20, 21, 22, 23], [24, 25, 26]]


In [15]:
padding_x = pad_sequences(x, max([len(i) for i in x])+1)
padding_x

array([[ 0,  0,  0,  1,  2],
       [ 0,  0,  0,  0,  3],
       [ 0,  0,  4,  5,  6],
       [ 0,  0,  0,  7,  8],
       [ 0,  9, 10, 11, 12],
       [ 0,  0, 13,  1, 14],
       [ 0,  0, 15, 16, 17],
       [ 0,  0,  0, 18, 19],
       [ 0, 20, 21, 22, 23],
       [ 0,  0, 24, 25, 26]], dtype=int32)

In [16]:
word_size = len(token.word_index) +1
word_size

27

In [36]:
Embedding(word_size, 8)

In [ ]:
for i in 

In [17]:
model = Sequential()
model.add(Embedding(word_size, 8))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['accuracy'])
model.fit(padding_x, classes, epochs=20)
print(model.evaluate(padding_x, classes))

Epoch 1/20


I0000 00:00:1747876261.387586    2090 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1347 MB memory:  -> device: 0, name: NVIDIA GeForce MX450, pci bus id: 0000:01:00.0, compute capability: 7.5
I0000 00:00:1747876262.080574    2139 service.cc:152] XLA service 0x7f2f98014620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747876262.080625    2139 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce MX450, Compute Capability 7.5
2025-05-22 10:11:02.104825: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1747876262.219950    2139 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.6000 - loss: 0.6932
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.7000 - loss: 0.6906
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.8000 - loss: 0.6879

I0000 00:00:1747876262.773927    2139 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.8000 - loss: 0.6879
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.9000 - loss: 0.6853
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.9000 - loss: 0.6827
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9000 - loss: 0.6800
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9000 - loss: 0.6774
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9000 - loss: 0.6748
Epoch 9/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.9000 - loss: 0.6721
Epoch 10/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9000 - loss: 0.6695
Epoch 11/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.9000 - loss: 0.6669
Epoch 12/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.9000 - loss: 0.6642
Epoch 13/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.9000 - loss: 0.6616
Epoch 14/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 1.0000 - loss: 0.6589
Epoch 15/